<a href="https://colab.research.google.com/github/learn2Pro/llama/blob/main/learnllama/lora_inject_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn

In [3]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 3.7 MB/s eta 0:00:00


- references
  - https://huggingface.co/docs/peft/developer_guides/low_level_api

In [4]:
import inspect
import torch
from peft import inject_adapter_in_model, LoraConfig
from torch import nn

In [17]:
class DummyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.embedding = nn.Embedding(100, 1000)
    self.linear = nn.Linear(1000, 1000)
    self.lm_head = nn.Linear(1000, 100)

  def forward(self, x):
    x = self.embedding(x)
    x = self.linear(x)
    x = self.lm_head(x)
    return x

In [14]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["linear"],
)
lora_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=64, target_modules={'linear'}, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [19]:
model = DummyModel()
model

DummyModel(
  (embedding): Embedding(100, 1000)
  (linear): Linear(in_features=1000, out_features=1000, bias=True)
  (lm_head): Linear(in_features=1000, out_features=100, bias=True)
)

In [9]:
type(model.linear)

torch.nn.modules.linear.Linear

In [26]:
dummy_inputs = torch.LongTensor([[0,1,2,3,4,5,6,7]])
dummy_inputs.shape,model(dummy_inputs).shape

(torch.Size([1, 8]), torch.Size([1, 8, 100]))

In [29]:
nn.Embedding(10, 5)(dummy_inputs)

tensor([[[ 0.1791, -0.9010, -1.0093,  1.4889, -0.2674],
         [-0.0299,  0.6660,  1.1523, -0.6777, -0.7593],
         [-0.3459,  0.1020,  0.1680, -0.2007,  1.1532],
         [ 2.0739, -1.6505,  0.0533,  0.2430,  1.0685],
         [ 0.1663, -0.4189, -1.0534,  1.0814, -1.3158],
         [ 1.3785, -1.0215, -0.3025,  0.5205,  0.6224],
         [-0.3105,  0.9892, -0.1884,  1.8926,  0.4840],
         [-1.4149,  0.5178, -0.6076,  0.6289,  0.4407]]],
       grad_fn=<EmbeddingBackward0>)

In [22]:
print(inspect.getsource(nn.Embedding))

class Embedding(Module):
    r"""A simple lookup table that stores embeddings of a fixed dictionary and size.

    This module is often used to store word embeddings and retrieve them using indices.
    The input to the module is a list of indices, and the output is the corresponding
    word embeddings.

    Args:
        num_embeddings (int): size of the dictionary of embeddings
        embedding_dim (int): the size of each embedding vector
        padding_idx (int, optional): If specified, the entries at :attr:`padding_idx` do not contribute to the gradient;
                                     therefore, the embedding vector at :attr:`padding_idx` is not updated during training,
                                     i.e. it remains as a fixed "pad". For a newly constructed Embedding,
                                     the embedding vector at :attr:`padding_idx` will default to all zeros,
                                     but can be updated to another value to be used as the pad